In [1]:
# log in to huggingface in command line
# huggingface-cli login in python scrips directory

In [2]:
from datasets import load_dataset
dataset = load_dataset("openai/openai_humaneval")

c:\Users\saraf\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#inspect dataset
print(dataset["test"][0])

{'task_id': 'HumanEval/0', 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert can

In [6]:
#Save to excel
import pandas as pd
df = pd.DataFrame(dataset["test"])
df.to_excel('HumanEval.xlsx', index=False)

In [7]:
import os
import json
import re

In [9]:
# Define base directories
base_dir = "processed_dataset"
dirs = {
    "functions": os.path.join(base_dir, "functions"),
    "tests": os.path.join(base_dir, "tests"),
    "descriptions": os.path.join(base_dir, "descriptions"),
}

# Create directories if they don't exist
for dir_path in dirs.values():
    os.makedirs(dir_path, exist_ok=True)


In [10]:
#add __init__.py files to tests and functions

# Process All Instances

Current issues:
1. loop blocks are not indented correctly in the test files
2. bit writing all functions when there's more than one function (helper functions).
3. some tests are empty (example: 34)

In [19]:
for i in range(len(dataset["test"])):
    # 0. Process a single instance
    dataset_instance = dataset["test"][i]
    task_id = dataset_instance["task_id"].replace("/", "_")  # Ensure valid filename
    function_name = dataset_instance["entry_point"]

    # 1. Extract the function definition and docstring
    prompt = dataset_instance["prompt"]
    function_definition = re.findall(r"def [^\n]*", prompt)[0]  # Extract the function definition (first line)
    docstring = re.findall(r'"""(.*?)"""', prompt, re.DOTALL)  # Extract docstring between triple quotes

    # 2. Save the function implementation (without the docstring)
    func_file_path = os.path.join(dirs["functions"], f"{task_id}.py")
    with open(func_file_path, "w") as func_file:
        #check for imports and add if present
        prompt_lines = dataset_instance["prompt"].splitlines()
        first_line = prompt_lines[0].strip()
        if(first_line.startswith("from") | first_line.startswith("import")):
            func_file.write(first_line + "\n")

        #write cannonical solution
        func_file.write(function_definition + "\n")
        func_file.write(dataset_instance["canonical_solution"] + "\n")

    # 3. Save the test cases
    # Convert check(candidate) to be compatible with pytest
    test_file_path = os.path.join(dirs["tests"], f"test_{task_id}.py")  # Use task_id for the filename
    with open(test_file_path, "w") as test_file:
        # Import the function from the corresponding module
        test_file.write(f"from functions.{task_id} import {function_name}\n\n")
    
        # Write the test function
        test_file.write(f"def test_{task_id}():\n")
        test_lines = dataset_instance["test"].splitlines()
        for line in test_lines[9:]:  # Skip the `def check(candidate):` line
            l = line.strip()
            mod = l.replace("candidate", function_name)
            test_file.write(f"  {mod}\n")  # Properly indent the lines
    
    # 4. Save the description (docstring)
    desc_file_path = os.path.join(dirs["descriptions"], f"{task_id}_description.txt")
    with open(desc_file_path, "w",encoding="utf-8") as desc_file:
        desc_file.write(f"Task ID: {dataset_instance['task_id']}\n")
        if docstring:
            desc_file.write(f"Description:\n{docstring[0].strip()}\n")  # Clean docstring without extra spaces
        else:
            desc_file.write("No description provided.\n")
        desc_file.write(f"Entry Point: {dataset_instance['entry_point']}\n")
    

# Compute Test Coverage

1. run python -m pytest --cov=functions --cov-report=term-missing > coverage_report.txt
2. run compute_cov.sh script

# Process One Instance

In [12]:
dataset_instance = dataset["test"][0]

# Process a single instance
task_id = dataset_instance["task_id"].replace("/", "_")  # Ensure valid filename
function_name = dataset_instance["entry_point"]

print(function_name)
print(task_id)

has_close_elements
HumanEval_0


In [13]:
# 1. Extract the function definition and docstring
prompt = dataset_instance["prompt"]
function_definition = re.findall(r"def [^\n]*", prompt)[0]  # Extract the function definition (first line)
docstring = re.findall(r'"""(.*?)"""', prompt, re.DOTALL)  # Extract docstring between triple quotes

#2 Current Issues:
1. (RESOLVED) not writing in function imports when needed

In [14]:
# 2. Save the function implementation (without the docstring)
func_file_path = os.path.join(dirs["functions"], f"{task_id}.py")
with open(func_file_path, "w") as func_file:
    #check for imports and add if present
    prompt_lines = dataset_instance["prompt"].splitlines()
    first_line = prompt_lines[0].strip()
    if(first_line.startswith("from") | first_line.startswith("import")):
        func_file.write(first_line + "\n")

    #write cannonical solution
    func_file.write(function_definition + "\n")
    func_file.write(dataset_instance["canonical_solution"] + "\n")

#3 Current Issues: 
1. not sure if writing in test case file in the most efficient way (ie: guessing on which line to start, changing the test to a different format than given in HumanEval)
2. not sure if current way of writing test case file is compilable with proper path setup

In [15]:
# 3. Save the test cases
# Convert check(candidate) to be compatible with pytest
test_file_path = os.path.join(dirs["tests"], f"test_{task_id}.py")  # Use task_id for the filename
with open(test_file_path, "w") as test_file:
    # Import the function from the corresponding module
    test_file.write(f"from functions.{task_id} import {function_name}\n\n")
    
    # Write the test function
    test_file.write(f"def test_{task_id}():\n")
    test_lines = dataset_instance["test"].splitlines()
    for line in test_lines[9:]:  # Skip the `def check(candidate):` line
        l = line.strip()
        mod = l.replace("candidate", function_name)
        test_file.write(f"  {mod}\n")  # Properly indent the lines


In [ ]:
# 4. Save the description (docstring)
desc_file_path = os.path.join(dirs["descriptions"], f"{task_id}_description.txt")
with open(desc_file_path, "w") as desc_file:
    desc_file.write(f"Task ID: {dataset_instance['task_id']}\n")
    if docstring:
        desc_file.write(f"Description:\n{docstring[0].strip()}\n")  # Clean docstring without extra spaces
    else:
        desc_file.write("No description provided.\n")
    desc_file.write(f"Entry Point: {dataset_instance['entry_point']}\n")